## Домашнее задание 11, SVM.
### Deadline -  29.11.2024    


## Основная часть

Для работы домашнего задания, необходимо загрузить файл svm.py по тому же пути, по которому лежит этот файл.

Эта домашняя работа будет отличаться от предыдущих и ее идея показать и рассказать как рабобоает SVM с разными методами нахождения оптимума. Поэтому в этой работе не будет заданий по написанию большой части кода или вывода двойстенных задач.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification, make_circles
pd.options.display.max_columns = 200
import svm
from svm import SVM, visualize
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Введение в SVM

Пусть перед нами стоит задачи банарной классификации. Будем считать, что $y_i \in \{ -1, 1\}; \quad y \in \mathbb{R}^d$.

Запишем нашу первоначальную задачу в виде задачи нахождения разделяющей гиперплоскости:
$$
    \begin{cases}
        \omega x_i + b \geqslant 1, \quad y_i = 1 \\
        \omega x_i + b < 1, \quad y_i = -1
    \end{cases}
$$.
Запишем задачу формально:
$$
    \sum_{i} \mathbb{I} \left[ y_i \not = sign( \langle \omega, x_i \rangle + b) \right] \rightarrow \min_{\omega}\\
    \sum_{i} \mathbb{I} \left[ y_i ( \langle \omega, x_i \rangle + b) < 0\right] \rightarrow \min_{\omega}
$$

Велечина $M = y_i ( \langle \omega, x_i \rangle + b)$ называется отступом классификатора. Легко увидеть, что 
1) Отступ положителен, когда $sign(y_i) = sing(\langle \omega, x_i \rangle + b)$, то есть класс угадан верно. При этом чем больше отступ, тем больше расстояние от $x_i$ до разделяющей гиперплоскости
2) Отступ отрицателен, когда $sign(y_i) \not = sing(\langle \omega, x_i \rangle + b)$, то есть класс угадан неверно. При этом чем больше отступ, тем больше ошибается классификатор.


В некоторых случаях гиперплоскость можно расположить несколькими способами. Поэтому возникает логичное желание не только найти разделяющую прямую, а еще и провести ее на одинаковом удалении от обоих классов, то есть максимизировать минимальный отступ.

Это можно сделать, положив функцию ошибки равной $max(0, 1 - M)$. Добавим к задаче $L_2$ регуляризацию и получим следующую задачу:
$$
    \frac{1}{2}||\omega||^2 + C\sum_{i} \max (0, 1 - y_i ( \langle \omega, x_i \rangle + b)) \rightarrow \min_{\omega}
$$

Также легко к этой задаче можно найти двойственную, что Вы уже проделывали в домашнем задании 9. На всякий случай запишем и двойственную задачу:
$$
    -\sum_{i=1}^d \lambda_i + \frac{1}{2}\sum_{i=1}^{d}\sum_{j=1}^{d} \lambda_i\lambda_j y_iy_j\langle x_i, x_j \rangle \rightarrow \min_{\lambda}\\
    s.t. \quad 0 \leqslant \lambda_i \leqslant C, \quad i = 1,...,d;\\
    \sum_{i=1}^d \lambda_iy_i = 0
$$

__Hint__: На самом деле эта задача двойственная к немного другой, если хотите строгого теоретического вывода, то посмотрите [презентацию К.В.Воронцова](http://machinelearning.ru/wiki/images/archive/a/a0/20150316112120!Voron-ML-Lin-SVM.pdf)

__Задача 1. (всего 2.5 балла)__ Сравнение различных способов нахождения решения линейных задачи.

В библиотеке реализованы следующие методы решения задачи SVM:
1) Метод внутренней точки для решения прямой задачи
2) Метод внутренней точки для решения двойственной задачи
3) Метод субградиентного спуска для решения прямой задачи
4) Метод стохастического субградиентного спуска для решения прямой задачи
5) Метод liblinear из библиотеки liblinear (решает задачи оптимизации путем последовательного выполнения приближенной минимизации вдоль координатных направлений или координатных гиперплоскостей)
6) Метод libsvm из библиотеки libsvm (решает задачу оптимизации путем разбиения задачи на ряд наименьших возможных подзадач, которые затем решаются аналитически)

__а). (1 балл)__ Сравнение временной сложности способов в зависимости от размерности. 

В этой задаче вам предстоит выбрать размерности, на которых будут сравниваться методы, дописать недостающих код и вывести на график зависимость времени работы от размерности задачи.

Рекомендуем выбрать как минимум 10 различных размерностей из отрезка [50, 1000].

In [ ]:
# инициализация различных методов, эти строки не нужно менять
Pclf = SVM(method='primal')
Dclf = SVM(method='dual', kernel='linear')
Sclf = SVM(method='subgradient')
StochSclf = SVM(method='stoch_subgradient')
LLclf = SVM(method='liblinear')
LSclf = SVM(method='libsvm', kernel='linear')
rng = np.random.default_rng(4202)

def generate_dataset(n_samples: int, dim: int, rnd = 4202):
    return make_classification(n_samples=n_samples, n_features=int(dim*1.2), n_informative=dim, random_state=rnd)

N = 300 # рекомендуемое количество n_samples в генерации датасета
D_array = 0 # размерности задач

# добавьте сохранение времени рана для каждого из методов
for D in D_array:
    X, y = generate_dataset(N, D)
    y = y[:, np.newaxis]

    # ниже код запуска самих алгоритмов, сами строки запуска не нужно менять,
    # но Вы можете дописывать нужный код между этих строк
    Pclf.fit(X,y)
    
    Dclf.fit(X,y)

    Sclf.fit(X,y)
    
    StochSclf.fit(X,y)
    
    LLclf.fit(X,y)
    
    LSclf.fit(X,y)
#

In [ ]:
# постройте график зависимости времени от размерности, добавьте на него все методы

__Ваше решение__ 

Опишите наблюдаемые на графиках результаты. Как думаете, почему они такие?

__б). (1.5 балла)__ Сравнение временной сложности способов в зависимости от количества элементов. 

В этой задаче вам предстоит выбрать количество элементов в датасете, на котором будут сравниваться методы, и вывести на график зависимость времени работы от количества элементов в датасете.

Рекомендуем выбрать как минимум 10 различных значений из отрезка [300, 1500].

In [ ]:
# инициализация различных методов, эти строки не нужно менять
Pclf = SVM(method='primal')
Dclf = SVM(method='dual', kernel='linear')
Sclf = SVM(method='subgradient')
StochSclf = SVM(method='stoch_subgradient')
LLclf = SVM(method='liblinear')
LSclf = SVM(method='libsvm', kernel='linear')


D = 200 # рекомендуемая размерность задачи
N_array = 0 # количество элементов в датасете

# добавьте сохранение времени рана для каждого из методов
for N in N_array:
    X, y = generate_dataset(N, D, 4202)
    y = y[:, np.newaxis]

    Pclf.fit(X,y)
    
    Dclf.fit(X,y)

    Sclf.fit(X,y)
    
    StochSclf.fit(X,y)
    
    LLclf.fit(X,y)
    
    LSclf.fit(X,y)
#

In [ ]:
# постройте график зависимости времени от количества элементов в датасете, добавьте на него все методы

__Ваше решение__ 

Опишите наблюдаемые на графиках результаты. Как думаете, почему они такие?

# Введение в решение нелинейных задач классификации.
Порой бывает так, что данное нам множество не может быть линейно отделимо, но нам все еще хочется использовать SVM для решения задачи. Как в таком случае быть? 

Нужно найти способ из нелинейного множества сделать линейное! В этом нам могут помочь преобразование в другое пространство (возможно с большей размерностью) с помощью ядер. Это помогает из изначальных данных сделать линейно-отделимые.

Приведем примеры нескольких ядер:
1) линейное - $K(x, y) = x^T y$
2) полиномиальное - $K(x, y) = (x^Ty + r)^p$, $\quad r \geqslant 0, q \geqslant 1$
3) радиальное базисное (RBF) - $K(x, y) = \exp(-\lambda ||x - y||^2)$, $\quad \lambda > 0$
4) сигмоидное $K(x, y) = \tanh (\beta_0 x^Ty + \beta_1)$

В этом случае двойственная задача решается следующим образом:

$$
    -\sum_{i=1}^d \lambda_i + \frac{1}{2}\sum_{i=1}^{d}\sum_{j=1}^{d} \lambda_i\lambda_j y_iy_j K(x_i, x_j) \rightarrow \min_{\lambda}\\
    s.t. \quad 0 \leqslant \lambda_i \leqslant C, \quad i = 1,...,d;\\
    \sum_{i=1}^d \lambda_iy_i = 0
$$

__Задача 2. (всего 2.5 балла)__ Сравнение различных способов нахождения решения нелинейных задачи.

В библиотеке реализованы следующие методы решения нелинейной задачи с использованием RBF ядра.
1) Метод внутренней точки для решения двойственной задачи
2) Метод libsvm из библиотеки libsvm (решает задачу оптимизации путем разбиения задачи на ряд наименьших возможных подзадач, которые затем решаются аналитически)

__Hint__ Т.к. задача генерации $n$-мерных шаров трудна, то будет сравнивать работу на линейной задачи. Это все равно не отражается на результатах.

__а). (1 балл)__ Сравнение временной сложности алгоритмов с RBF ядром в зависимости от размерности. 

В этой задаче вам предстоит выбрать размерности, на которых будут сравниваться методы, и вывести на график зависимость времени работы от размерности задачи.

Рекомендуем выбрать как минимум 10 различных размерностей из отрезка [10, 2000].

In [ ]:
# инициализация различных методов, эти строки не нужно менять
Dclf = SVM(method='dual', kernel='rbf', gamma=0.1)
LSclf = SVM(method='libsvm', kernel='rbf', gamma=0.1)


N = 300 # рекомендуемое количество n_samples в генерации датасета
D_array = 0 # размерность задачи

# добавьте сохранение времени рана для каждого из методов
for D in D_array:
    X, y = generate_dataset(N, D)
    y = y[:, np.newaxis]
    
    # ниже код запуска самих алгоритмов, сами строки запуска не нужно менять,
    # но Вы можете дописывать нужный код между этих строк
    Dclf.fit(X, y)
    
    LSclf.fit(X, y)

In [ ]:
# постройте график зависимости времени от размерности, добавьте на него все методы

__Ваше решение__ 

Опишите наблюдаемые на графиках результаты. Как думаете, почему они такие?

Также сравните результаты с решением линейной задачи без использования ядра.

__б). (1.5 балла)__ Сравнение временной сложности решения задач c RBF ядром в зависимости от количества элементов. 

В этой задаче вам предстоит выбрать количество элементов в датасете, на котором будут сравниваться методы, и вывести на график зависимость времени работы от количества элементов в датасете.

Рекомендуем выбрать как минимум 10 различных значений из отрезка [300, 1500].

In [ ]:
Dclf = SVM(method='dual', kernel='rbf', gamma=0.1)
LSclf = SVM(method='libsvm', kernel='rbf', gamma=0.1)



D = 200 # рекомендуемая размерность задачи
N_array = 0 # количество элементов в датасете

# добавьте сохранение времени рана для каждого из методов
for N in N_array:
    X, y = generate_dataset(N, D)
    y = y[:, np.newaxis]
    
    Dclf.fit(X, y)
    
    LSclf.fit(X, y)

In [ ]:
# постройте график зависимости времени от количества элементов в датасете, добавьте на него все методы

__Ваше решение__ 

Опишите наблюдаемые на графиках результаты. Как думаете, почему они такие?

Также сравните результаты с решением линейной задачи без использования ядра.

## Дополнительная часть

Рассмотрим двойстеннцю задачу с RBF ядром подробнее, запишем ее:

$$
    -\sum_{i=1}^d \lambda_i + \frac{1}{2}\sum_{i=1}^{d}\sum_{j=1}^{d} \lambda_i\lambda_j y_iy_j \exp(-\lambda ||x - y||^2) \rightarrow \min_{\lambda}\\
    s.t. \quad 0 \leqslant \lambda_i \leqslant C, \quad i = 1,...,d;\\
    \sum_{i=1}^d \lambda_iy_i = 0
$$

Как можно видеть, есть два гиперпараметра - $C, \lambda$. В этой части задания мы подберем лучшие значения этих гиперпараметров для двух задач.

__Задача 1. (всего 5 баллов)__ Подбор гиперпараметров для хорошо и плохо разделимой выборки. Сравнение результатов.

### Хорошо разделимая выборка

In [ ]:
# генерация датасета, раскомментируйте код
'''
N = 800
D = 2
batch = N // 4
dataset = np.zeros((N, D))

np.random.seed(42)

dataset[:, 0][:batch] = np.random.normal(loc=3, scale=1.2, size=batch)
dataset[:, 1][:batch] = np.random.normal(loc=3, scale=1.2, size=batch)
dataset[:, 0][batch: 2 * batch] = np.random.normal(loc=-3, scale=1.2, size=batch)
dataset[:, 1][batch: 2 * batch] = np.random.normal(loc=-3, scale=1.2, size=batch)
dataset[:, 0][2 * batch: 3 * batch] = np.random.normal(loc=3, scale=1.2, size=batch)
dataset[:, 1][2 * batch: 3 * batch] = np.random.normal(loc=-3, scale=1.2, size=batch)
dataset[:, 0][3 * batch:] = np.random.normal(loc=-3, scale=1.2, size=batch)
dataset[:, 1][3 * batch:] = np.random.normal(loc=3, scale=1.2, size=batch)
target = -np.ones((N, 1), dtype=np.int32)
target[2 * batch:, 0] = 1

plt.close()
plt.scatter(dataset[:, 0], dataset[:, 1], c=target, cmap=plt.cm.jet)
plt.title('Model Dataset', fontsize=16)
plt.xlabel('x1', fontsize=16)
plt.ylabel('x2', fontsize=16)
plt.show()
'''

__а). (1 балл)__ Допишите недостающий код. Функция должна сохранять лучшее занчение mean accuracy, а также гиперпараметры gamma и C для этого лучшего значения.

In [ ]:
def BruteSearchCV(X, y, method, C_list, gamma_list):
    best_mean_accuracy = -1.0
    best_gamma = -1.0
    best_C = -1.0
    results = []
    cv = KFold(n_splits=7, shuffle=True, random_state=42)
    for gamma in gamma_list:
        for C in C_list:
            clf = svm.SVM(C=C, method=method, kernel='rbf', gamma=gamma)
            acc_list = []
            for train_index, test_index in cv.split(X, y):
                X_train, X_test = X[train_index, :], X[test_index, :]
                y_train, y_test = y[train_index, :], y[test_index, :]
                clf.fit(X_train, y_train)
                acc_list.append(accuracy_score(y_true=y_test, y_pred=clf.predict(X_test, return_classes=True).T[0]))
            mean_acc_score = np.array(acc_list).mean()
            if mean_acc_score > best_mean_accuracy:
                # Ваш код, после написания удалите pass
            results.append({'mean_accuracy': , 'gamma': , 'C': })
    return {'best_mean_accuracy': best_mean_accuracy, 'best_gamma': best_gamma, 'best_C': best_C}, results

__б). (1 балл)__ Допишите недостающий код запуска поиска с методами 'dual' и 'libsvm'.

In [ ]:
# ран ячейки может занять до 10 минут
gamma_list = 10.0**np.arange(-5, 6)
C_list = 10.0**np.arange(-5, 6)
dual_best, dual_table = # BruteSearchCV для 'dual'
libsvm_best, libsvm_table = # BruteSearchCV для 'libsvm'

In [ ]:
clf = SVM(method='libsvm', C=libsvm_best['best_C'], kernel='rbf', gamma=libsvm_best['best_gamma'])
clf.fit(dataset, target)
visualize(dataset, target, clf, show_vectors=True, title='Model Dataset decision regions',
          return_classes=False)

In [ ]:
dual_table = pd.DataFrame(dual_table)
libsvm_table = pd.DataFrame(libsvm_table)

plt.close()
plt.figure(figsize=(6.3, 5))
xx2 = dual_table.gamma
xx1 = dual_table.C
accuracy_mesh = dual_table.mean_accuracy
sc = plt.scatter(xx1, xx2, c=accuracy_mesh, cmap=plt.cm.jet, s=740, vmin=0.0, vmax=1.0, marker='s')
plt.title('Dependence of accuracy from C and gamma', fontsize=12)
plt.xlabel('C', fontsize=12)
plt.ylabel('gamma', fontsize=12)
plt.xlim([C_list.min(), C_list.max()])
plt.ylim([gamma_list.min(), gamma_list.max()])
plt.xscale('log')
plt.yscale('log')
plt.colorbar(sc)
print('Results for dual method:')
plt.show()

### Плохо разделимая выборка

In [ ]:
# генерация датасета, раскомментируйте код
'''
N = 800
D = 2

np.random.seed(42)

means = np.zeros(D)
covs = 2 * np.diag(np.ones(D))
dataset = np.random.multivariate_normal(means, covs, N)
batch = N // 2
target = np.random.choice([1, -1], size=N)[:, np.newaxis]
plt.close()
plt.scatter(dataset[:, 0], dataset[:, 1], c=target, cmap=plt.cm.jet)
plt.title('Model Dataset', fontsize=16)
plt.xlabel('x1', fontsize=16)
plt.ylabel('x2', fontsize=16)
plt.show()
'''

In [ ]:
# ран ячейки может занять до 10 минут
gamma_list = 10.0**np.arange(-5, 6)
C_list = 10.0**np.arange(-5, 6)
dual_best, dual_table = BruteSearchCV(dataset, target, 'dual', C_list, gamma_list)
libsvm_best, libsvm_table = BruteSearchCV(dataset, target, 'libsvm', C_list, gamma_list)
table = {'dual': dual_best, 'libsvm': libsvm_best}

In [ ]:
clf = SVM(method='libsvm', C=libsvm_best['best_C'], kernel='rbf', gamma=libsvm_best['best_gamma'])
clf.fit(dataset, target)
visualize(dataset, target, clf, show_vectors=True, title='Model Dataset decision regions',
          return_classes=False)

In [ ]:
dual_table = pd.DataFrame(dual_table)
libsvm_table = pd.DataFrame(libsvm_table)

plt.close()
plt.figure(figsize=(6.3, 5))
xx2 = dual_table.gamma
xx1 = dual_table.C
accuracy_mesh = dual_table.mean_accuracy
sc = plt.scatter(xx1, xx2, c=accuracy_mesh, cmap=plt.cm.jet, s=740, vmin=0.0, vmax=1.0, marker='s')
plt.title('Dependence of accuracy from C and gamma', fontsize=12)
plt.xlabel('C', fontsize=12)
plt.ylabel('gamma', fontsize=12)
plt.xlim([C_list.min(), C_list.max()])
plt.ylim([gamma_list.min(), gamma_list.max()])
plt.xscale('log')
plt.yscale('log')
plt.colorbar(sc)
print('Results for dual method:')
plt.show()

__в). (3 балла)__ Посмотрите на зависмость точности от различных $C$ и $\gamma$. Напишите почему так происходит, подкрепите свои рассуждения необходимыми выкладками.  

__Ваше решение__